In [74]:
#cell: init driver

import undetected_chromedriver.v2 as uc
options = uc.ChromeOptions()

dirname = os.path.abspath('')
profilePath = dirname + r'\profileUpwork'
options.add_argument(r'--user-data-dir=' + profilePath)

options.add_argument('--no-first-run --no-service-autorun --password-store=basic')
driver = uc.Chrome(options=options)  

driver.get("https://www.upwork.com") 
driver.maximize_window()
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import pickle
import sys
sys.setrecursionlimit(100000) #is set to allow pickle work with larger files


class TagData: #all data about tag is stored in this class
    def __init__(self,tagName,entry,intermediate,expert,prop5,prop510,prop1015,prop1520,prop2050,previousClients,paymentVerified,hourly,fixedPrice,fp100,
                 fp100500,fp5001k,fp1k5k,fp5k,projLen1,projLen13,projLen36,projLen6,hoursLess30,hoursMore30,hires0,hires19,
                 hires10,connect2,connect4,connect6):
        self.tagName = tagName
        self.entry = entry
        self.intermediate = intermediate
        self.expert = expert
        self.prop5 = prop5
        self.prop510 = prop510
        self.prop1015 = prop1015
        self.prop1520 = prop1520
        self.prop2050 = prop2050
        self.previousClients = previousClients
        self.paymentVerified = paymentVerified
        self.hourly = hourly
        self.fixedPrice = fixedPrice
        self.fp100 = fp100
        self.fp100500 = fp100500
        self.fp5001k = fp5001k
        self.fp1k5k = fp1k5k
        self.fp5k = fp5k
        self.projLen1 = projLen1
        self.projLen13 = projLen13
        self.projLen36 = projLen36
        self.projLen6 = projLen6
        self.hoursLess30 = hoursLess30
        self.hoursMore30 = hoursMore30
        self.hires0 = hires0
        self.hires19 = hires19
        self.hires10 = hires10
        self.connect2 = connect2
        self.connect4 = connect4
        self.connect6 = connect6

class customNewButton(object): #custom wait in google login for next window to load
  def __init__(self, element):
    self.element = element

  def __call__(self, driver):
    buttonNext2 = driver.find_element(By.XPATH , "//*/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/button")
    if buttonNext2.id != self.element.id:
        return buttonNext2
    else:
        return False

class customReload(object): #custom wait until results load after search input
  def __init__(self, oldEntry,oldIntermediate,oldExpert):
    self.oldEntry = oldEntry
    self.oldIntermediate = oldIntermediate
    self.oldExpert = oldExpert

  def __call__(self, driver):
    newEntry = driver.find_element(By.XPATH , "//*[@id='main']/div/div/div/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/fieldset/div[1]/label/span[2]/small")
    newIntermediate = driver.find_element(By.XPATH , '//*[@id="main"]/div/div/div/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/fieldset/div[2]/label/span[2]/small')
    newExpert = driver.find_element(By.XPATH , '//*[@id="main"]/div/div/div/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/fieldset/div[3]/label/span[2]/small')
    if (newEntry.text != self.oldEntry or newIntermediate.text != self.oldIntermediate or newExpert.text != self.oldExpert
    or (newEntry.text == 0 and newIntermediate.text == 0 and newExpert.tex == 0)):
        return True
    else:
        return False

In [ ]:
#function per page approach to handle captcha exceptions.
#cell: colecting data

def clickWithOffset(element):
    ActionChains(driver).move_to_element_with_offset(element, 1 , 1) \
    .move_by_offset(int(-element.location['x']/2),int(-element.location['y']/2)) \
    .move_to_element_with_offset(element, int(element.size['width']/2) , int(element.size['height']/2)).click().perform()

def clickWithOffset2(element):
    action = ActionChains(driver)
    print(1)
    action.move_to_element_with_offset(element, 1 , 1).perform()
    print(2)
    action.move_by_offset(int(-element.location['x']/2),int(-element.location['y']/2)) .perform()
    print(3)
    action.move_to_element_with_offset(element, int(element.size['width']/2) , int(element.size['height']/2)).perform()
    print(4)
    action.click().perform()


def mainPage():
    print("main")
    global finishedGlobal
    goToSearchPage = WebDriverWait(driver, 10).until( 
        EC.presence_of_element_located((By.XPATH,'//button[@data-test="job-search-button"]')))
    print('main found')
    clickWithOffset2(goToSearchPage)
    finishedGlobal = True

def searchPage():
    
    searchNotActive = WebDriverWait(driver, 10).until( 
        EC.presence_of_element_located((By.XPATH,'//input[@placeholder="Search for jobs"]')))
    clickWithOffset2(searchNotActive)

    search = WebDriverWait(driver, 10).until( 
        EC.presence_of_element_located((By.XPATH,'//input[@placeholder="Search for jobs"]')))
    searchButton = driver.find_element(By.XPATH , '//button[@data-test="job-search-button"]')
    print(searchButton.location)
    print(searchButton.size)

    with open("savedTagsList", "rb") as f:   
        myStoredList = pickle.load(f)

    time.sleep(1)
    global resultsGlobal
    global startAtGlobal
    global finishedGlobal
    upTo = 30 #len(myStoredList)
    for i in myStoredList[startAtGlobal:upTo]:
        oldEntry = driver.find_element(By.XPATH , '//div[@data-test="filter-sidebar-component_contractorTier"]/div/div[2]/div/fieldset/div[1]/label/span[2]/small')
        oldEntryStr = oldEntry.text #need to make string variable because elements get reloaded.
        oldIntermediate = driver.find_element(By.XPATH , '//div[@data-test="filter-sidebar-component_contractorTier"]/div/div[2]/div/fieldset/div[2]/label/span[2]/small')
        oldIntermediateStr = oldIntermediate.text
        oldExpert = driver.find_element(By.XPATH , '//div[@data-test="filter-sidebar-component_contractorTier"]/div/div[2]/div/fieldset/div[3]/label/span[2]/small')
        oldExpertStr = oldExpert.text
        print(i)
        search.send_keys(Keys.CONTROL,'a')
        search.send_keys(i)
        time.sleep(.1)
        clickWithOffset2(searchButton)
        WebDriverWait(driver, 15).until(customReload(oldEntryStr,oldIntermediateStr,oldExpertStr))
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')

        filters = soup.find(attrs={"data-test": "filters-sidebar"})
        counts = filters.find_all('small',attrs={"class": "text-muted"})
        l = []
        s = 's'
        myTable = s.maketrans('','','()')
        for c in counts:
            l.append(int(c.string.translate(myTable)))
        o = TagData(i,*l)
        resultsGlobal.append(o)
        startAtGlobal+=1
    finishedGlobal = True

def exceptionHandler(func):
    while(True):
        print('loop')
        start_time = time.time()
        try:
            func()
        except Exception as e:
            print(e.__class__.__name__)
            print('Program stopped probably because of captcha. Complete captcha in browser, then type any input in this window and press enter, and program will continue.')
            a = input()
        if (finishedGlobal):
            print("time passed ", time.time() - start_time)
            break 
            
startAtGlobal = 0
resultsGlobal = []
finishedGlobal = False
exceptionHandler(mainPage)

startAtGlobal = 0
resultsGlobal = []
finishedGlobal = False
exceptionHandler(searchPage)


In [ ]:
searchButton.location

In [ ]:
print(searchButton.location)
print(searchButton.size)

In [89]:
#mini program: M1 scrape one page and collect tags and compare them with exsiting list and display new found ones
#set up: open webdriver and go to search page. Use categories 'data entry and..', 'all- data science', 'database managment',
#'all-web,mobile...'
source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')
import re
tags = soup.find_all('a',attrs={"class": re.compile("up-skill-badge text-muted")})
foundTags = []
for t in tags:
    foundTags.append(t.string)
foundTags = list(set(foundTags))
newTags = []
with open("savedTagsList", "rb") as f:   
    myStoredList = pickle.load(f)
for f in foundTags:
    if f not in myStoredList:
        newTags.append(f)
newTagsWithoutUnwanted = []
with open("savedUnwantedTags", "rb") as f:   
    storedUnwanted = pickle.load(f)
for n in newTags:
    if n not in storedUnwanted:
        newTagsWithoutUnwanted.append(n)
print(newTagsWithoutUnwanted)

['Adobe AIR', 'iPad App Development', 'IBM WebSphere', 'Socket.io', 'Custom Web Design', 'Oracle WebLogic Server', 'WildFly', 'Flash Builder', 'Poster Design', 'Zoho Creator', 'Navigation System Design', 'Websockets', 'Apache Geronimo', 'Growth Hacking', 'Corona', 'Eclipse Jetty', 'Oracle GlassFish Server']


In [111]:
#mini program: M2  .Fill list l with tags found in M1 mini program. They will pe added to stored tag list.
l = ['Shopify']
lNoDups = list(set(l))
with open("savedTagsList", "rb") as f:   
    myStoredList = pickle.load(f)
myStoredList.extend(lNoDups)
with open("savedTagsList", "wb") as f:   
    pickle.dump(myStoredList, f)

In [87]:
#mini program: M3 .Save found unwanted tags to file of unwanted tags. Fill l list from output of M1 mini program, when it was 
#reran after running M2 mini program
l = ['Shopify']

with open("savedUnwantedTags", "rb") as f:   
    storedUnwanted = pickle.load(f)
storedUnwanted.extend(l)
with open("savedUnwantedTags", "wb") as f:   
    pickle.dump(storedUnwanted, f)

In [162]:
#mini program: M4. print out results list so that it can be safely remade in another cell and saved to file without crashing.
#used when i want to save scraped data.
for r in resultsGlobal:
    print('tempList.append(TagData("'+ r.tagName +'",int('+str(r.entry)+'),int('+str(r.intermediate)+'),int('+str(r.expert)+'),int('+str(r.prop5)+'),int('
         +str(r.prop510)+'),int('+str(r.prop1015)+'),int('+str(r.prop1520)+'),int('+str(r.prop2050)+'),int('+str(r.previousClients)+'),int('
         +str(r.paymentVerified)+'),int('+str(r.hourly)+'),int('+str(r.fixedPrice)+'),int('+str(r.fp100)+'),int('+str(r.fp100500)+'),int('
         +str(r.fp5001k)+'),int('+str(r.fp1k5k)+'),int('+str(r.fp5k)+'),int('+str(r.projLen1)+'),int('+str(r.projLen13)+'),int('
         +str(r.projLen36)+'),int('+str(r.projLen6)+'),int('+str(r.hoursLess30)+'),int('+str(r.hoursMore30)+'),int('+str(r.hires0)+'),int('
         +str(r.hires19)+'),int('+str(r.hires10)+'),int('+str(r.connect2)+'),int('+str(r.connect4)+'),int('+str(r.connect6)+')))')


##this code is for saving data into file. Coppy this code into another cell, then paste into it output of this cell, and run it.
#with open("savedTagsData", "rb") as f:   
#    storedTagsData = pickle.load(f)
#tempList = []
##paste print here  
#storedTagsData.extend(tempList)
#with open("savedTagsData", "wb") as f:   
#    pickle.dump(storedTagsData, f)


tempList.append(TagData("Oracle Database Administration",int(4),int(25),int(34),int(17),int(13),int(3),int(7),int(19),int(0),int(39),int(38),int(25),int(8),int(9),int(1),int(5),int(2),int(5),int(7),int(7),int(19),int(26),int(12),int(30),int(13),int(20),int(9),int(26),int(28)))
tempList.append(TagData("WordPress",int(1564),int(8492),int(5070),int(1622),int(2499),int(2151),int(1754),int(4868),int(0),int(10150),int(9159),int(5971),int(1998),int(2167),int(774),int(839),int(193),int(3969),int(2818),int(865),int(1507),int(8100),int(1059),int(6142),int(3649),int(5339),int(6192),int(4260),int(4677)))
tempList.append(TagData("Unit Testing",int(25),int(378),int(342),int(122),int(119),int(108),int(96),int(210),int(0),int(545),int(588),int(157),int(24),int(26),int(26),int(46),int(35),int(73),int(96),int(66),int(353),int(259),int(329),int(280),int(180),int(285),int(98),int(170),int(477)))
tempList.append(TagData("API Integration",int(208),int(2515),int(2197),int(628),int(1026),int(777),int(602),int

In [147]:
#mini program: M5. Give data scores and print out tags sorted by best scores. Lowest score is best. Score represents position in sorted list.

class tagScores:
    def __init__(self,tag):
        self.tag = tag 
        self.scores = {}

with open("savedTagsData", "rb") as f:   
    storedTagsData = pickle.load(f)

tagsScores = []
for t in storedTagsData:
    tagsScores.append(tagScores(t))

##sort by job amounts
def sortAmounts(o):
    return o.tag.entry + o.tag.intermediate + o.tag.expert
tagsScores.sort(reverse=True, key=sortAmounts)
i=1
for t in tagsScores:
    t.scores["all amounts"] = i
    i+=1

##sort by entry job amounts
def sortEntry(o):
    return o.tag.entry
tagsScores.sort(reverse=True, key=sortEntry)
i=1
for t in tagsScores:
    t.scores["entry amounts"] = i
    i+=1

#print out scores
def sortScores(o):
    return sum(o.scores.values())
tagsScores.sort(key=sortScores)
for t in tagsScores:
    print(t.tag.tagName, sum(t.scores.values()), t.scores)

Website 4 {'all amounts': 2, 'entry amounts': 2}
Web Development 4 {'all amounts': 1, 'entry amounts': 3}
Data Entry 6 {'all amounts': 5, 'entry amounts': 1}
WordPress 9 {'all amounts': 4, 'entry amounts': 5}
HTTPS 10 {'all amounts': 6, 'entry amounts': 4}
JavaScript 10 {'all amounts': 3, 'entry amounts': 7}
Web Service 15 {'all amounts': 9, 'entry amounts': 6}
App Development 16 {'all amounts': 8, 'entry amounts': 8}
PHP Script 23 {'all amounts': 10, 'entry amounts': 13}
Mobile App 26 {'all amounts': 17, 'entry amounts': 9}
Mobile App Development 29 {'all amounts': 19, 'entry amounts': 10}
HTML 30 {'all amounts': 13, 'entry amounts': 17}
Email Automation 30 {'all amounts': 12, 'entry amounts': 18}
API 30 {'all amounts': 7, 'entry amounts': 23}
PHP 33 {'all amounts': 14, 'entry amounts': 19}
CSS 35 {'all amounts': 15, 'entry amounts': 20}
Data Management 37 {'all amounts': 21, 'entry amounts': 16}
Python 38 {'all amounts': 16, 'entry amounts': 22}
Testing 39 {'all amounts': 27, 'entry 

In [67]:
#cell: init driver old version, where i dont use undetected_chromedriver


# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.service import Service

# option = webdriver.ChromeOptions()
# option.add_argument('--disable-blink-features=AutomationControlled')
# import os
# dirname = os.path.abspath('')
# profilePath = dirname + r'\profileUpwork'
# option.add_argument(r'--user-data-dir=' + profilePath)
# #option.headless = True
# path = ChromeDriverManager().install()
# try:
#     with open(path, "rb") as f: #need to use driver quit to free up file handle when rerunning program.  
#         filedata = f.read()
#     filedata = filedata.replace(b'$cdc_asdjflasutopfhvcZLmcfl_',b'$btlhsaxJbTXmBATUDvTRhvcZLm_')
#     with open(path, "wb") as f:   
#         f.write(filedata)
# except:
#     pass

# driver = webdriver.Chrome(service=Service(path),options=option)
# driver.get("https://www.upwork.com")
# driver.maximize_window()
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
# from bs4 import BeautifulSoup
# import time
# import pickle
# import sys
# sys.setrecursionlimit(100000) #is set to allow pickle work with larger files


[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\write\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache


In [97]:
#not used anymore, because i now use browser profile, and login happens automatically.
#cell: navigate to search page
#How to navigate to search page manualy: login, then just click the button with magnifying glass symbol on the right of 
#search input field.

# accept = WebDriverWait(driver, 10).until( 
#     EC.presence_of_element_located((By.ID,"onetrust-accept-btn-handler"))
# )
# time.sleep(2)
# accept.click()
# login = WebDriverWait(driver, 5).until( 
#     EC.presence_of_element_located((By.XPATH,"//a[@class='nav-item login-link d-none d-lg-block px-20']"))
# )
# time.sleep(5)
# login.click()
# googleConnect = WebDriverWait(driver, 5).until( 
#     EC.presence_of_element_located((By.XPATH,'//*[@id="login_google_submit"]'))
# )
# time.sleep(5)
# googleConnect.click()

# WebDriverWait(driver, 5).until(EC.number_of_windows_to_be(2))
# driver.switch_to.window(driver.window_handles[1])

# email = driver.find_element(By.XPATH , "/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div/div[1]/div/div[1]/input")
# time.sleep(5)
# email.send_keys('fakem7506@gmail.com')

# buttonNext = driver.find_element(By.XPATH , "/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/button")
# time.sleep(5)
# buttonNext.click()

# password = WebDriverWait(driver, 5).until( 
#     EC.presence_of_element_located((By.XPATH , "/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div[1]/div/div/div/div/div[1]/div/div[1]/input"))
# )
# time.sleep(5)
# password.send_keys('zaidimas8')

# buttonNext2 = WebDriverWait(driver, 5).until( 
#     customNewButton(buttonNext)
# )
# time.sleep(5)
# buttonNext2.click()

# WebDriverWait(driver, 5).until(EC.number_of_windows_to_be(1))
# driver.switch_to.window(driver.window_handles[0])

# suggestionClose = WebDriverWait(driver, 15).until( 
#     EC.presence_of_element_located((By.XPATH,'//*[@id="main"]/div/div/aside/div/div[1]/div[1]/section/div[2]/div[2]/div/div/div/div[2]/button'))
# )
# suggestionClose.click()

# search = driver.find_element(By.XPATH , '//button[@data-test="job-search-button"]')
# search.click()

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00385FD3+2187219]
	Ordinal0 [0x0031E6D1+1763025]
	Ordinal0 [0x00233E78+802424]
	Ordinal0 [0x00261C10+990224]
	Ordinal0 [0x00261EAB+990891]
	Ordinal0 [0x0028EC92+1174674]
	Ordinal0 [0x0027CBD4+1100756]
	Ordinal0 [0x0028CFC2+1167298]
	Ordinal0 [0x0027C9A6+1100198]
	Ordinal0 [0x00256F80+946048]
	Ordinal0 [0x00257E76+949878]
	GetHandleVerifier [0x006290C2+2721218]
	GetHandleVerifier [0x0061AAF0+2662384]
	GetHandleVerifier [0x0041137A+526458]
	GetHandleVerifier [0x00410416+522518]
	Ordinal0 [0x00324EAB+1789611]
	Ordinal0 [0x003297A8+1808296]
	Ordinal0 [0x00329895+1808533]
	Ordinal0 [0x003326C1+1844929]
	BaseThreadInitThunk [0x76876739+25]
	RtlGetFullPathName_UEx [0x76F78FEF+1215]
	RtlGetFullPathName_UEx [0x76F78FBD+1165]


In [11]:
#failed approach to handle captchas on every driver action. Fails because webpage gets reloaded and elements get lost.
#cell: colecting data
#different behaviour: if search field wasnt clicked then 4 lines about searchNotActive should be uncommented, but if it was clicked then should comment them.

# def captchaHandlerResult(elementSearch):
#     try:
#         result = elementSearch()
#     except:
#         a = input()
#         result = elementSearch()
#     finally:
#         return result
# def captchaHandler(action):
#     try:
#         action()
#     except:
#         a = input()
#         action()
# goToSearchPage = captchaHandlerResult(lambda : WebDriverWait(driver, 10).until( 
#     EC.presence_of_element_located((By.XPATH,'//button[@data-test="job-search-button"]')))
# )
# captchaHandler(lambda : goToSearchPage.click())
# time.sleep(2) #wait for captcha to pop up

# searchNotActive = captchaHandlerResult(lambda : WebDriverWait(driver, 10).until( 
#     EC.presence_of_element_located((By.XPATH,'//*[@id="main"]/div/div/div/div/div[2]/div/div[2]/div/section/div[1]/div[1]/div[1]/div/div/div/div/input')))
# )
# captchaHandler(lambda : searchNotActive.click())

# search = captchaHandlerResult(lambda : WebDriverWait(driver, 10).until( 
#     EC.presence_of_element_located((By.XPATH,'//*[@id="main"]/div/div/div/div/div[2]/div/div[2]/div/section/div[1]/div[1]/div[1]/div/span/div/div[1]/div/input')))
# )
# searchButton = captchaHandlerResult(lambda : driver.find_element(By.XPATH , '//*[@id="main"]/div/div/div/div/div[2]/div/div[2]/div/section[1]/div[1]/div[1]/div[2]/button'))


# with open("savedTagsList", "rb") as f:   
#     myStoredList = pickle.load(f)

# results = []
# time.sleep(1)
# startAt = 0
# upTo = 2#len(myStoredList)
# for i in myStoredList[startAt:upTo]:
#     oldEntry = captchaHandlerResult(lambda : driver.find_element(By.XPATH , "//*[@id='main']/div/div/div/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/fieldset/div[1]/label/span[2]/small"))
#     oldEntryStr = oldEntry.text #need to make string variable because elements get reloaded.
#     oldIntermediate = captchaHandlerResult(lambda : driver.find_element(By.XPATH , '//*[@id="main"]/div/div/div/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/fieldset/div[2]/label/span[2]/small'))
#     oldIntermediateStr = oldIntermediate.text
#     oldExpert = captchaHandlerResult(lambda : driver.find_element(By.XPATH , '//*[@id="main"]/div/div/div/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/fieldset/div[3]/label/span[2]/small'))
#     oldExpertStr = oldExpert.text
#     print(i)
#     captchaHandler(lambda : search.send_keys(Keys.CONTROL,'a'))
#     captchaHandler(lambda : search.send_keys(i))
#     time.sleep(1)
#     captchaHandler(lambda : searchButton.click())
#     captchaHandlerResult(lambda : WebDriverWait(driver, 15).until( 
#         customReload(oldEntryStr,oldIntermediateStr,oldExpertStr))
#     )
#     source = driver.page_source
#     soup = BeautifulSoup(source, 'html.parser')

#     filters = soup.find(attrs={"data-test": "filters-sidebar"})
#     counts = filters.find_all('small',attrs={"class": "text-muted"})
#     l = []
#     s = 's'
#     myTable = s.maketrans('','','()')
#     for c in counts:
#         l.append(int(c.string.translate(myTable)))
#     o = TagData(i,*l)
#     results.append(o)

In [ ]:
driver.quit()